In [2]:
import pandas as pd
import boto3
import zipfile
import os
from tqdm import tqdm
import shutil

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# AWS Stuff

In [4]:
destination = '/content/gdrive/My Drive/safegraph-patterns' #enter your desired Google Drive folder without a '/' at the end

accessKey = 'RDT348CQNBUUIXW8ITTT'
secretKey = 'vYeorvQZdFnS6FM2ADXN8tNzzbydEfuJcXZfbFp3'

session = boto3.Session(
    aws_access_key_id=accessKey, # Found at: https://catalog.safegraph.io/app/browse -- under start here and 'Reveal Access Key'
    aws_secret_access_key=secretKey, # Found at: https://catalog.safegraph.io/app/browse -- under start here and 'Reveal Access Key'
    region_name = 'us-east-1' # Note this used to be us-west-2 --- it has changed to us-east-1
)

bucket = 'sg-c19-response'
prefix = 'monthly-patterns/'
s3 = session.client('s3', endpoint_url = 'https://s3.wasabisys.com')

# Functions

In [5]:
def getCorrespondingDate(key):
  if 'csv' in key:
    if 'backfill' in key:
      date = '-'.join(key.split("/")[6:8])
      return(date)
    else:
      split = key.split("/")[2:4]
      date = '-'.join([split[0], str(int(split[1])-1).zfill(2)])
      return(date)
  return("")

def addEntry(date_dict, key):
  date = getCorrespondingDate(x['Key'])
  if date != "":
    if date in date_dict.keys():
      date_dict[date] += [key]
    else:
      date_dict[date] = [key]
  return(date_dict)

def filterCombineDel(files):
  keep = []
  for f in tqdm(files):
    for chunk in tqdm(pd.read_csv(f, chunksize=100000)):
      #TODO: HERE IS WHERE YOU SET THE SPECIFIC FILTERS YOU WANT FOR YOUR DATA
      df = chunk[chunk.region.isin(['FL'])]
      keep.append(df)
    deleteFile(f)
  return(pd.concat(keep))

def deleteFile(fp):
  open(fp, 'w').close() #rewrite the file to contain zero data to save storage
  os.remove(fp) #send file to trash, user can delete it from there if they want but it takes up no storage
  return(True)

# Make Dictionary of Files

In [6]:
d = s3.list_objects(Bucket=bucket, Prefix=prefix)
date_dict = {}
for x in d['Contents']:
  date_dict = addEntry(date_dict, x['Key'])

In [7]:
date_dict

{'2018-01': ['monthly-patterns/home_panel_summary_backfill/2020/05/07/12/2018/01/home_panel_summary.csv',
  'monthly-patterns/normalization_stats_backfill/2020/05/07/12/2018/01/normalization_stats.csv',
  'monthly-patterns/patterns_backfill/2020/05/07/12/2018/01/patterns-part1.csv.gz',
  'monthly-patterns/patterns_backfill/2020/05/07/12/2018/01/patterns-part2.csv.gz',
  'monthly-patterns/patterns_backfill/2020/05/07/12/2018/01/patterns-part3.csv.gz',
  'monthly-patterns/patterns_backfill/2020/05/07/12/2018/01/patterns-part4.csv.gz',
  'monthly-patterns/visit_panel_summary_backfill/2020/05/07/12/2018/01/visit_panel_summary.csv'],
 '2018-02': ['monthly-patterns/home_panel_summary_backfill/2020/05/07/12/2018/02/home_panel_summary.csv',
  'monthly-patterns/normalization_stats_backfill/2020/05/07/12/2018/02/normalization_stats.csv',
  'monthly-patterns/patterns_backfill/2020/05/07/12/2018/02/patterns-part1.csv.gz',
  'monthly-patterns/patterns_backfill/2020/05/07/12/2018/02/patterns-part2.c

# Download, filter, combine and delete

In [8]:
months = ['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2020-07']

In [10]:
for x in tqdm(months):
  if x not in os.listdir(destination):
    os.mkdir(destination + "/" + x)
  for f in date_dict[x]:
    if f.split('/')[-1] not in os.listdir(destination + '/' + x): #do not download the file if it is already there
      s3.download_file(bucket, f, '/'.join([destination, x, f.split('/')[-1]]))
  files = ['/'.join([destination, x, f]) for f in os.listdir('/'.join([destination, x])) if 'patterns' in f]
  df = filterCombineDel(files)
  df.to_csv("/".join([destination, x, "patterns.csv"]))

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

1it [00:04,  4.75s/it]

2it [00:09,  4.76s/it]

3it [00:13,  4.60s/it]

4it [00:17,  4.40s/it]

5it [00:21,  4.24s/it]

6it [00:25,  4.18s/it]

7it [00:29,  4.15s/it]

8it [00:33,  4.09s/it]

9it [00:37,  4.01s/it]

10it [00:41,  3.95s/it]

11it [00:43,  3.93s/it]

 25%|██▌       | 1/4 [00:43<02:10, 43.44s/it]

0it [00:00, ?it/s]

1it [00:03,  3.95s/it]

2it [00:07,  3.94s/it]

3it [00:11,  3.95s/it]

4it [00:15,  3.93s/it]

5it [00:19,  3.91s/it]

6it [00:23,  3.90s/it]

7it [00:27,  3.89s/it]

8it [00:31,  3.88s/it]

9it [00:35,  3.87s/it]

10it [00:38,  3.86s/it]

11it [00:40,  3.72s/it]

 50%|█████     | 2/4 [01:24<01:25, 42.72s/it]

0it [00:00, ?it/s]

1it [00:03,  3.94s/it]

2it [00:07,  3.95s/it]

3it [00:11,  3.96s/it]

4it [00:15,  3.95s/it]

5it [00:19,  3.95s/it]

6it [00:23,  3.93s/it]

7it [00:27,  3.91s/it]

8it [00:31,  3.89s/it]

9it [00:35,  3.90s/it]

10it [00:39,  3.88s/it]

11it [00:41,  3.74s/it]

 75%|████

# Create smaller output files

In [11]:
for x in months:
  df = pd.read_csv(destination + '/' + x + '/patterns.csv')
  df1 = df[['safegraph_place_id', 'poi_cbg', 'visitor_home_cbgs']]
  df1.to_csv(destination + '/output/' + 'patterns-' + x + '.csv')
  df2 = df[['safegraph_place_id', 'poi_cbg', 'visits_by_day', 'raw_visit_counts', 'raw_visitor_counts']]
  df2.to_csv(destination + '/output/' + 'patterns-vbd-' + x + '.csv')

In [12]:
for x in months:
  df = pd.read_csv(destination + '/' + x + '/home_panel_summary.csv')
  df = df[df.state == 'fl']
  df.to_csv(destination + '/output/' + 'hp-' + x + '.csv')

# Scrap code

In [ ]:
x = months[0]
if x not in os.listdir(destination):
  os.mkdir(destination + "/" + x)
for f in date_dict[x]:
  if f.split('/')[-1] not in os.listdir(destination + '/' + x): #do not download the file if it is already there
    s3.download_file(bucket, f, '/'.join([destination, x, f.split('/')[-1]]))

In [ ]:
files = ['/'.join([destination, x, f]) for f in os.listdir('/'.join([destination, x])) if 'patterns' in f]
files

['/content/gdrive/My Drive/safegraph-patterns/2020-01/patterns-part1.csv.gz',
 '/content/gdrive/My Drive/safegraph-patterns/2020-01/patterns-part2.csv.gz',
 '/content/gdrive/My Drive/safegraph-patterns/2020-01/patterns-part3.csv.gz',
 '/content/gdrive/My Drive/safegraph-patterns/2020-01/patterns-part4.csv.gz']

In [ ]:
df = filterCombineDel(files)
df.to_csv("/".join([destination, x, "patterns.csv"]))

In [ ]:
df1 = df[['safegraph_place_id', 'poi_cbg', 'visitor_home_cbgs']]
df1.to_csv("/content/gdrive/My Drive/safegraph-patterns/patterns-2020-01.csv")